In [1]:
from torchvision import transforms, utils
from scripts import PostdamCarsDataModule, Skeleton, fid_score, DynamicPad, AddNoise
import PIL

In [7]:
tr =  transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                                 DynamicPad(min_img_dim=(130, 70),
                                            padding_mode="constant", padding_value=125),
                                 transforms.RandomRotation(
                                     degrees=5, resample=PIL.Image.NEAREST, fill=125),
                                 transforms.RandomCrop(
                                     (60, 120), padding_mode="reflect"),
                                 transforms.Resize((32, 64)),
                                 transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomVerticalFlip(p=0.5),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5], [0.5]),
                                 AddNoise(alpha=0.07)])

pots = PostdamCarsDataModule("../potsdam_data/cem-v0/v2/training_tightcanvas_graybackground",
                             batch_size=64, 
                             transform=tr,
                             img_size=(32, 64))

pots.setup()
data = pots.train_dataloader()

car, _ = next(iter(data))
grid = utils.make_grid(car, normalize=True, pad_value=-1)
utils.save_image(grid, "test1.png")

In [ ]:
tr = transforms.Compose([transforms.Resize((32, 64)),
                         Skeleton(1, 20, True, True),
                         transforms.RandomRotation(
                             degrees=5, resample=PIL.Image.NEAREST, fill=255),
                         #transforms.RandomAffine(degrees=0, 
                         #                        translate=(0.05, 0.1), 
                         #                        shear=0, 
                         #                        scale=(0.9, 0.95),
                         #                        fillcolor=255, 
                         #                        resample=PIL.Image.NEAREST),
                         #                       
                         transforms.ToTensor(),
                         transforms.RandomHorizontalFlip(p=0.5),
                         transforms.RandomVerticalFlip(p=0.5),
                         transforms.Normalize([0.5], [0.5])])

pots = PostdamCarsDataModule("../potsdam_data/gta_cars_online_masked_cars",
                             batch_size=64, 
                             transform=tr,
                             img_size=(48, 96))

pots.setup()
data = pots.train_dataloader()

car, _ = next(iter(data))
grid = utils.make_grid(car, normalize=True, pad_value=1)
utils.save_image(grid, "gta_cars.png")

In [ ]:
help(transforms.RandomAffine)

In [ ]:
img_dim = (3, 32, 64)
data_dir1 = "../potsdam_data/potsdam_cars_all"
data_dir2 = "../potsdam_data/artificial_cars"

transform1 = transforms.Compose([transforms.Resize(img_dim[1:]),
                                transforms.ToTensor(),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomVerticalFlip(p=0.5),
                                transforms.ColorJitter(hue=[-0.1, 0.1]),
                                transforms.Normalize([0.5], [0.5])])
transform2 = transforms.Compose([transforms.Resize(img_dim[1:]),
                                 transforms.ToTensor(),
                                 transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomVerticalFlip(p=0.5),
                                 transforms.Normalize([0.5], [0.5])])

potsdam = PostdamCarsDataModule(
    data_dir1, img_size=img_dim[1:], batch_size=1024,
    data_dir2=data_dir2, transform=transform1, transform2=transform2)
potsdam.setup()
data = potsdam.train_dataloader()

fids = []
for _ in range(5):
    real, artificial = next(iter(data))
    fids.append(fid_score(real, artificial, device="cuda:0"))
np.mean(fids)

In [ ]:
np.mean(fids), np.std(fids)

In [ ]:
act1 = np.squeeze(vgg16_get_activation_maps(real, layer_idx=33, device="cpu", normalize_range=(-1, 1)).numpy())
act2 = np.squeeze(vgg16_get_activation_maps(artificial, layer_idx=33, device="cpu", normalize_range=(-1, 1)).numpy())
fid_score(act1, act2, skip_vgg=True)

In [ ]:
act1 = vgg16_get_activation_maps(real, layer_idx=33, device="cpu", normalize_range=(-1, 1))
act2 =  vgg16_get_activation_maps(artificial, layer_idx=33, device="cpu", normalize_range=(-1, 1))
fid_score(act1, act2, skip_vgg=True)

In [ ]:
import os
import cv2

folder = "../edges2shoes/train"
files = os.listdir(folder)

shoes = "../edges2shoes/train_shoes"
edges = "../edges2shoes/train_edges"
os.makedirs(shoes, exist_ok=True)
os.makedirs(edges, exist_ok=True)

for file in files[:5000]:
    if os.path.splitext(file)[-1] == ".jpg":
        img = cv2.imread(os.path.join(folder, file))
        e = img[:, :256]
        s = img[:, 256:]
        
        cv2.imwrite(os.path.join(shoes, file), s)
        cv2.imwrite(os.path.join(edges, file), e)
        
        
        
        

In [ ]:

tr = transforms.Compose([transforms.Resize((96, 96)),
                         transforms.RandomRotation(
                             degrees=5, resample=PIL.Image.NEAREST, fill=255),
                         transforms.ToTensor(),
                         transforms.RandomHorizontalFlip(p=0.5),
                         transforms.Normalize([0.5], [0.5])])

pots = PostdamCarsDataModule("../edges2shoes/train_shoes",
                             data_dir2="../edges2shoes/train_edges",
                             batch_size=64, 
                             transform=tr,
                             transform2=tr,
                             img_size=(64, 64))

pots.setup()
data = pots.train_dataloader()

car, edge = next(iter(data))
grid = utils.make_grid(edge, normalize=True)
utils.save_image(grid, "edge2car.png")

In [5]:
from torchvision import transforms, utils
from scripts import PostdamCarsDataModule

pots = PostdamCarsDataModule("../potsdam_data/potsdam_cars_corrected",
                             data_dir2="../potsdam_data/potsdam_cars_corrected_failed",
                             batch_size=30)

pots.setup()
data = pots.train_dataloader()

passed, failed = next(iter(data))

grid = utils.make_grid(passed, nrow=10, normalize=True, pad_value=1)
utils.save_image(grid, "passed_cars.png")

grid = utils.make_grid(failed, nrow=10, normalize=True, pad_value=1)
utils.save_image(grid, "failed_cars.png")

In [21]:
from torchvision import transforms, utils
from scripts import PostdamCarsDataModule, fid_score

pots = PostdamCarsDataModule("../potsdam_data/potsdam_cars_corrected",
                             data_dir2="../potsdam_data/cem-v0/v2/training_tightcanvas_graybackground",
                             transform = transforms.Compose([transforms.Resize((32, 64)),
                                                                transforms.ToTensor(),
                                                                transforms.RandomHorizontalFlip(p=0.5),
                                                                transforms.RandomVerticalFlip(p=0.5),
                                                                transforms.ColorJitter(
                                                                    hue=[-0.1, 0.1], contrast=[1, 1.25]),
                                                                transforms.Normalize([0.5], [0.5])]),
                             transform2 = transforms.Compose([transforms.Resize((32, 64)),
                                                                transforms.ToTensor(),
                                                                transforms.RandomHorizontalFlip(p=0.5),
                                                                transforms.RandomVerticalFlip(p=0.5),
                                                                transforms.Normalize([0.5], [0.5])]),
                             batch_size=1024)


pots.setup()
data = pots.train_dataloader()

fid_scores = []
for i in range(5):
    real1, artificial = next(iter(data))
    real2, artificial = next(iter(data))
    fid_scores.append(fid_score(real1, real2, device="cuda:0"))

max(fid_scores), min(fid_scores), sum(fid_scores) / 5

(1.3456232109880308, 1.3209501295767494, 1.3297460703722241)

In [20]:
fid_scores

[11.548408550171875,
 11.71328583857035,
 11.544662657979536,
 11.796701227101828,
 11.716654176712044]

In [3]:
from torchvision import transforms, utils
from scripts import PostdamCarsDataModule

pots = PostdamCarsDataModule("experiments/dcgan_experiments/version_2/results/images/epoch=974",
                             data_dir2="experiments/dcgan_experiments/version_2/results/images/epoch=24",
                             batch_size=30)

pots.setup()
data = pots.train_dataloader()

passed, failed = next(iter(data))

grid = utils.make_grid(passed, nrow=10, normalize=True, pad_value=1)
utils.save_image(grid, "best_fid_cars.png")

grid = utils.make_grid(failed, nrow=10, normalize=True, pad_value=1)
utils.save_image(grid, "worst_fid_cars.png")

In [5]:
from torchvision import transforms, utils
from scripts import PostdamCarsDataModule

pots = PostdamCarsDataModule("../potsdam_data/potsdam_cars_corrected",
                             data_dir2="../potsdam_data/potsdam_cars_corrected",
                             shuffle=False,
                             transform = transforms.Compose([transforms.Resize((32, 64)),
                                                                transforms.ToTensor(),
                                                                transforms.Normalize([0.5], [0.5])]), 
                             transform2 = transforms.Compose([transforms.Resize((32, 64)),
                                                                transforms.ToTensor(),
                                                                transforms.ColorJitter(
                                                                    hue=[-0.1, 0.1], contrast=[1, 1.25]),
                                                                transforms.Normalize([0.5], [0.5])]),                             
                             batch_size=30)

pots.setup()
data = pots.train_dataloader()

original, data_aug = next(iter(data))

grid = utils.make_grid(original, nrow=10, normalize=True, pad_value=1)
utils.save_image(grid, "figs/original_cars.png")

grid = utils.make_grid(data_aug, nrow=10, normalize=True, pad_value=1)
utils.save_image(grid, "figs/data_aug_cars.png")